# Red Neuronal con Keras
Los pasos para poder crear nuestro modelo, son los siguientes:

1. Cargar datos.
2. Definir tu Modelo.
3. Compilar el Modelo.
4. Entrenar el Modelo.
5. Evaluar el Modelo.
6. Unir todos los pasos anteriores.

## Datasets

Vamos a utilizar uno de los datasets disponible por la Universidad de California Irvine, en este caso: **Diabetes de Indios Pima**. Este dataset es un conjunto de datos disponible de manera gratuita desde el repositorio UCI Machine Learning. Describe los datos de registros médicos de los pacientes y describe si ellos tuvieron diabetes dentro de un periodo de cinco años. Es un problema de clasificación binaria (aparición de diabetes como 1 o no como 0). Las variables de entrada que describen a cada paciente son numéricas y tienen escalas variables. A continuación se enumeran los ocho atributos para el conjunto de datos:

1. Número de veces embarazada.
2. Concentración de glucosa plasmática a 2 horas en una prueba oral de tolerancia a la glucosa.
3. Presión arterial diastólica (mm Hg).
4. Grosor del pliegue de la piel del tríceps (mm).
5. Insulina en suero de 2 horas (mu U / ml).
6. Índice de masa corporal.
7. Función de pedigrí de diabetes.
8. Edad (años).
9. Clase, aparición de diabetes dentro de los cinco años.

Debido a que todos los atributos son numericos, es facil utilizarlos directamente en la red neuronal. Este dataset contiene 768 registros.
La precisión base de todas las predicciones que se realizan es del 65.1%. Los mejores resultados se encuentran en el rango de 77.7% de precisión utilizando 10-fold cross validation.

## Cargar datos

En el siguiente código, se importan las librerias de Keras y numpy.
Debido a que todos los datos son numéricos, se puede importar el archivo directo con numpy.
También se inicia el generador aleatorio con la variable seed, para que siempre obtengamos los mismos resultados y no desviados.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

FILENAME = '../diabetes.csv'

seed = 7
numpy.random.seed(seed)

dataset = numpy.loadtxt(FILENAME, delimiter=',')

training_data = dataset[:, 0:8]
training_targets = dataset[:, 8]

## Definir tu modelo

Los modelos en Keras son definidos como una sequencias de capas. Podemos crear un modelo sequencia (**Sequencial**) y agregar capas una a una hasta que cumplan nuestros requerimientos.

El primer paso es asegurarnos que la primer capa tenga el numero correcto de entradas. Estos puede ser especificado con el argumento **input_dim**, en este caso utilizaremos el valor de 8 debido a nuestras 8 variables de entrada.

*¿Cómo saber el numero de capas y sus tipos?* Esta es una pregunta complicada pero normalmente se experimenta y a prueba y error se puede llegar a un resultado óptimo. En este caso vamos a utilizar una red conectada completamente con 3 capas.

Capas completamente conectadas son definidas mediante la clase **Dense**. En esta clase se define el numero de neuronas como primer argumento, como segundo argumento se define el método de inicialización, y la función de activación. En este caso vamos a iniciar los pesos de la red con un valor aleatorio muy pequeño utilizando una distribución uniforme (uniform), en Keras el valor va de 0 a 0.05. Otra opción seria utilizar una distribución de Gauss (normal).

En las primeras dos capas se definirá una función de activación relu y para la capa de salida una función sigmoid. En el pasado las funciones sigmoid y tanh eran las utilizadas en todas las capas, pero ha sido demostrado que el desempeño mejora utilizando la función rectifier como activación. En nuestro caso utilizamos la función sigmoid en la capada de salida para asegurarnos que el valor de salido oscila entre 0 y 1 y es mas fácil definir el resultado en la clasificación binaria.

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

## Compilar modelo

Una vez que el modelo esta definido, se procede a **compilarlo**. Al compilar el modelo se mandan llamar las librerias en el backend, en nuestro caso Tensorflow. En este caso el backend automáticamente selecciona la mejor forma de representar la red neuronal para entrenamiento y realizar predicciones en el hardware. Cuando se compila el modelo, se deben definir algunas propiedades adicionales requeridas para el entrenamiento del modelo:
- Especifica la función de perdida o loss function, que es utilizada para evaluar los pesos.
- También debes definir el optimizador utilizado para buscar entre los pesos de la red y algunas métricas opcionales que se require colectar y reportar durante el entrenamiento.
En este ejemplo utilizaremos una función de perdida **binary_crossentropy**, que es una función logarítmica de perdida. Se utilizará una función para los calcular los gradientes llamada **adam**. Al ser un problema de clasificación binaria, se coleccionará y reportará la precision de la clasificación utilizando **accuracy**.

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Entrenar el modelo

Una vez que este definido y compilado tu modelo, es tiempo de **ejecutar el modelo con los dato**s. Podemos entrenar nuestro modelo llamando la **función fit()** en el modelo.

El proceso de entrenamiento se ejecuta cierto numero de veces utilizando el dataset, este numero de veces es llamado **epochs**, y se define utilizando el parámetro nb_epoch. También podemos definir el numero de instances que son evaluadas antes de que los pesos sean actualizados en la red neuronal. Este parámetro se llama batch_size. En este problema se definirá un numero pequeño de epochs (150) y un valor relativamente pequeño de el batch (10).

In [ ]:
model.fit(training_data, training_targets, epochs=150, batch_size=10)

## Evalúa el modelo

Ahora que hemos entrenado nuestro modelo utilizando el dataset completo, debemos evaluar el rendimiento de la red neuronal. Estos nos da una idea que tan bien modelamos nuestro dataset, pero no tendremos idea como se desempeñara el modelo con datos nuevos. En la realidad se debe de separar el dataset de training y el dataset de testing.

Puedes evaluar tu modelo en tu dataset de training utilizando la función **evaluation**() de tu modelo, pasándole la misma entrada y salida utilizada cuando lo entrenaste. **Esto generará una predicción por cada registro, se podrán colectar scores, average loss y otras métricas como accuracy.**

In [ ]:
scores = model.evaluate(training_data, training_targets)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))